## Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

## Load model and allocate tensors

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

## Make Detections

In [26]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    _,frame=cap.read()
    
    #Reshape Image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image = tf.cast(img,dtype=tf.float32)
    
    #setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    #Make predictions
    interpreter.set_tensor(input_details[0]['index'],np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    #Rendering
    draw_connections(frame,keypoints_with_scores,EDGES,0.4)
    draw_keypoints(frame,keypoints_with_scores,0.4)
    cv2.imshow('MoveNet Lightning',cv2.flip(frame,1))
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[[[0.74418944 0.4861698  0.54875106]
   [0.66618884 0.56270146 0.48591852]
   [0.6912521  0.44944426 0.7721274 ]
   [0.7059805  0.6793715  0.54990506]
   [0.72002023 0.4366783  0.52707934]
   [0.86976933 0.7079632  0.26573265]
   [0.8648857  0.33752394 0.42144695]
   [0.85485184 0.7328571  0.00745615]
   [0.93841827 0.286892   0.01006603]
   [0.70205843 0.70736355 0.03699416]
   [0.8527514  0.4128875  0.02309865]
   [0.51933545 0.67121005 0.04062864]
   [0.63016784 0.3739407  0.0150921 ]
   [1.0216383  0.63266337 0.0028773 ]
   [0.8616153  0.27789947 0.00678492]
   [0.99866045 0.02623299 0.01031595]
   [0.26466215 0.41198057 0.03157452]]]]
[[[[ 0.74830806  0.48837802  0.5434013 ]
   [ 0.6698894   0.56009626  0.52108485]
   [ 0.69534326  0.4502091   0.7025486 ]
   [ 0.6984538   0.67471904  0.6598781 ]
   [ 0.7205833   0.43571264  0.5366891 ]
   [ 0.87766165  0.7248339   0.21687958]
   [ 0.85453886  0.33782464  0.43299538]
   [ 0.8486337   0.7515268   0.01055083]
   [ 0.9360096   0.2810

[[[[0.76651454 0.59513795 0.6301675 ]
   [0.69244426 0.653993   0.59933144]
   [0.7023286  0.5235845  0.3433794 ]
   [0.7210972  0.70750797 0.5537923 ]
   [0.7361531  0.44972655 0.65710604]
   [0.8900668  0.74753654 0.21005306]
   [0.84791803 0.3442834  0.44404158]
   [0.8487911  0.8399316  0.0077039 ]
   [0.9321791  0.3137076  0.01005027]
   [0.76370305 0.72546625 0.01177004]
   [0.862638   0.41374582 0.01171723]
   [0.53522193 0.6597717  0.03044602]
   [0.6005037  0.39020014 0.04179597]
   [0.7242765  0.9660977  0.01512778]
   [0.8430097  0.32099164 0.02238187]
   [0.741879   0.75003886 0.00527903]
   [0.2672623  0.4121173  0.03940573]]]]
[[[[0.7671974  0.59244406 0.63892496]
   [0.6934203  0.64955604 0.5629445 ]
   [0.70462203 0.52462304 0.36873555]
   [0.7180788  0.70713127 0.56036127]
   [0.7355937  0.45318216 0.6004599 ]
   [0.88568586 0.74978274 0.18368894]
   [0.8481454  0.34582978 0.3950395 ]
   [0.848626   0.81678987 0.00693473]
   [0.9314269  0.3311935  0.00804633]
   [0.758

[[[[0.7738174  0.55361295 0.49957597]
   [0.6912358  0.6219735  0.6115261 ]
   [0.70941114 0.4978542  0.30929625]
   [0.7206112  0.7053138  0.57089764]
   [0.73388886 0.45659396 0.56290823]
   [0.8868119  0.73857516 0.21791425]
   [0.86598766 0.3606803  0.3624593 ]
   [0.8769977  0.9680667  0.01823047]
   [0.8489591  0.31269908 0.01865065]
   [0.7620832  0.73129195 0.02821472]
   [0.8352672  0.48853344 0.02797556]
   [0.7796781  0.6637764  0.01713499]
   [0.85022825 0.44699234 0.01552591]
   [1.0169675  0.75514764 0.00423199]
   [0.84646595 0.32022643 0.03620109]
   [0.8161495  0.5349479  0.00531194]
   [0.8154198  0.47681803 0.01328745]]]]
[[[[0.76726145 0.55008745 0.5675947 ]
   [0.6852573  0.619599   0.502952  ]
   [0.70911473 0.49743092 0.47425658]
   [0.71682894 0.7062358  0.5568399 ]
   [0.73280656 0.45568213 0.55076116]
   [0.88312435 0.73714036 0.19934648]
   [0.8596219  0.34530282 0.3926636 ]
   [0.8606051  0.85102355 0.00572115]
   [0.848686   0.31243172 0.02121407]
   [0.716

[[[[0.77392876 0.58256227 0.43304247]
   [0.69749975 0.6450851  0.5258697 ]
   [0.7124764  0.51884866 0.37842166]
   [0.71973574 0.7086121  0.61227256]
   [0.7350373  0.4598698  0.53343314]
   [0.88321275 0.7360008  0.24426278]
   [0.85453534 0.3575912  0.34632206]
   [0.856986   0.8548447  0.00599504]
   [0.93218243 0.34976003 0.0083594 ]
   [0.78519833 0.7326006  0.01342267]
   [0.83943814 0.49533543 0.02682784]
   [0.67572606 0.70278376 0.01588398]
   [0.6558536  0.44765973 0.0154359 ]
   [1.0190153  0.79758024 0.00543329]
   [0.84738815 0.31850442 0.03033298]
   [0.77474403 0.6830764  0.00406644]
   [0.26855764 0.41586897 0.03745514]]]]
[[[[0.7710437  0.58926123 0.44652867]
   [0.6967108  0.6504351  0.60112035]
   [0.70648396 0.521623   0.33405808]
   [0.7258382  0.71492165 0.5628673 ]
   [0.7383625  0.45844862 0.52649456]
   [0.88276875 0.7407347  0.22899479]
   [0.84985244 0.34484094 0.3956219 ]
   [0.8552784  0.83550996 0.00415775]
   [0.93019885 0.35061142 0.00707996]
   [0.753

[[[[0.7750238  0.5812611  0.41219547]
   [0.6990978  0.6457209  0.55005   ]
   [0.71519035 0.5190067  0.45373675]
   [0.7205226  0.71458256 0.6542467 ]
   [0.7395896  0.46090278 0.5608599 ]
   [0.88599986 0.7405707  0.22458741]
   [0.8596508  0.36251813 0.31709638]
   [0.8364501  0.96550155 0.01822439]
   [0.93256783 0.34977812 0.00773302]
   [0.75507677 0.7314963  0.01233289]
   [0.83845776 0.48030823 0.0223169 ]
   [0.55433494 0.648861   0.03892848]
   [0.64177394 0.44900975 0.01416659]
   [1.0188625  0.77499056 0.0054976 ]
   [0.8476948  0.3186149  0.03171089]
   [0.69867176 0.7260941  0.01081443]
   [0.26564568 0.41434354 0.04088029]]]]
[[[[0.7734114  0.59067404 0.51747596]
   [0.6984117  0.6532188  0.64272   ]
   [0.7110969  0.5238587  0.40434775]
   [0.72702956 0.7180408  0.57008064]
   [0.7438759  0.4567444  0.51359445]
   [0.8901176  0.7535163  0.18202978]
   [0.8584115  0.3582655  0.28528693]
   [0.83630514 0.9678172  0.01734623]
   [0.9322369  0.35044134 0.00682017]
   [0.717

...

[[[[0.3548839  0.37625012 0.5145066 ]
   [0.30525154 0.43480572 0.59483665]
   [0.32116753 0.32321644 0.5302146 ]
   [0.32049513 0.5038042  0.35298204]
   [0.33889136 0.27764004 0.5503711 ]
   [0.5002966  0.6980579  0.2064794 ]
   [0.554566   0.18633793 0.49105573]
   [0.5551126  0.8311372  0.15909058]
   [0.608284   0.12478175 0.0431594 ]
   [0.53305185 0.82211393 0.10382736]
   [0.5900003  0.12427512 0.32733232]
   [0.64554226 0.8078133  0.09447682]
   [0.73238814 0.7864791  0.06875411]
   [0.59052855 0.8282068  0.09006846]
   [0.6284535  0.27863872 0.00521144]
   [0.53917634 0.840044   0.05080682]
   [0.41671535 0.31103215 0.02524853]]]]
[[[[0.38509402 0.43234146 0.7501354 ]
   [0.33651716 0.47208363 0.826977  ]
   [0.3384828  0.3791432  0.7040353 ]
   [0.35319003 0.5104888  0.7549605 ]
   [0.3692648  0.31032884 0.6084147 ]
   [0.50792885 0.6175548  0.7868297 ]
   [0.55435    0.1750553  0.43477368]
   [0.6335818  0.9295299  0.6751067 ]
   [0.8400831  0.02384715 0.21195307]
   [0.678

[[[[0.3959307  0.51359004 0.5550713 ]
   [0.33742768 0.55411285 0.7336381 ]
   [0.3428859  0.45885262 0.6534974 ]
   [0.34476417 0.5909482  0.7637327 ]
   [0.35869536 0.38535517 0.7807993 ]
   [0.51107264 0.70440024 0.7325335 ]
   [0.52556694 0.2885186  0.91688967]
   [0.80255264 0.82907236 0.66370755]
   [0.79926085 0.18860918 0.40191293]
   [0.8635069  0.7484957  0.20673892]
   [0.8452141  0.23132251 0.13551605]
   [0.9216744  0.6201523  0.05653009]
   [0.918903   0.34848914 0.06100592]
   [0.81930304 0.80570924 0.03732586]
   [0.82266235 0.34376007 0.08068696]
   [0.84945095 0.6771083  0.01357633]
   [0.8402493  0.5755539  0.01223654]]]]
[[[[0.41252598 0.5101478  0.7465358 ]
   [0.35969576 0.5461659  0.6734688 ]
   [0.36865705 0.46092412 0.7278697 ]
   [0.36574864 0.5808493  0.70982873]
   [0.38233265 0.39338267 0.6007613 ]
   [0.5245562  0.6918355  0.90824515]
   [0.53563404 0.29988754 0.69477475]
   [0.78168297 0.7995872  0.62002975]
   [0.79085696 0.20650563 0.5293566 ]
   [0.853

[[[[0.43383345 0.5028766  0.63480127]
   [0.3860826  0.5383954  0.78522635]
   [0.39408344 0.4568922  0.5696655 ]
   [0.40060526 0.58097184 0.40725613]
   [0.41781348 0.401236   0.6925578 ]
   [0.55242926 0.68359935 0.70826983]
   [0.56013066 0.3192303  0.79613507]
   [0.80160165 0.75193083 0.40421355]
   [0.8187697  0.26273558 0.41759944]
   [0.84261954 0.6535126  0.18929246]
   [0.81742877 0.44165003 0.09429857]
   [0.9167709  0.61332107 0.04987949]
   [0.9287424  0.39034125 0.04185781]
   [0.7725069  0.75492954 0.09400174]
   [0.817765   0.36189562 0.1294257 ]
   [0.84731007 0.65746367 0.01917794]
   [0.8332453  0.46968848 0.0409728 ]]]]
[[[[0.43143877 0.5052159  0.720288  ]
   [0.38436338 0.5408141  0.7355554 ]
   [0.39241612 0.45867452 0.6303836 ]
   [0.3984129  0.58194196 0.4104755 ]
   [0.41545427 0.403206   0.76408815]
   [0.5466579  0.6840305  0.87878984]
   [0.5560018  0.32463515 0.7347696 ]
   [0.78933114 0.7523382  0.44794697]
   [0.8104695  0.2636547  0.3950959 ]
   [0.838

[[[[0.56765544 0.62483823 0.5639205 ]
   [0.50887877 0.6611754  0.8061088 ]
   [0.5095347  0.5624101  0.6612308 ]
   [0.5288627  0.68609214 0.5320589 ]
   [0.54273987 0.4723078  0.6210396 ]
   [0.73519564 0.79542315 0.7831067 ]
   [0.7353859  0.3377758  0.7471981 ]
   [0.8775996  0.9684912  0.03139859]
   [0.8553479  0.2592112  0.06073251]
   [0.74751365 0.7372884  0.01416692]
   [0.86370456 0.14922418 0.04115298]
   [0.49152303 0.8068132  0.01415116]
   [0.3492192  0.7639105  0.02291411]
   [0.78310144 0.9713634  0.01434445]
   [0.78620684 0.42161644 0.00740278]
   [0.7805879  1.0169593  0.00650522]
   [0.8427393  0.02009137 0.01948157]]]]
[[[[0.5533898  0.62268376 0.6292069 ]
   [0.4980747  0.6609827  0.64929146]
   [0.49905056 0.5603409  0.528004  ]
   [0.5141988  0.6839668  0.7191284 ]
   [0.52930176 0.46979356 0.8027656 ]
   [0.72365844 0.7991146  0.8375417 ]
   [0.7186998  0.3380539  0.7798449 ]
   [0.8637811  0.86520433 0.01044327]
   [0.8437681  0.2776584  0.13219318]
   [0.742

[[[[5.11437297e-01 5.52994668e-01 7.50657201e-01]
   [4.35117245e-01 5.95632076e-01 6.00033998e-01]
   [4.46980238e-01 4.78646725e-01 6.28975511e-01]
   [4.34456408e-01 6.37818098e-01 6.70864284e-01]
   [4.65654731e-01 3.96143645e-01 4.84407216e-01]
   [6.46915913e-01 7.75669992e-01 7.97101080e-01]
   [6.62132144e-01 2.78476059e-01 6.94539249e-01]
   [8.80032003e-01 9.20700252e-01 2.72051394e-02]
   [8.91537845e-01 1.46769583e-01 3.12073827e-02]
   [8.02766502e-01 9.75377798e-01 3.07398438e-02]
   [9.22016263e-01 4.15839314e-01 1.78539753e-03]
   [5.15430987e-01 6.07785344e-01 4.95657325e-03]
   [3.68540823e-01 3.30696404e-01 6.52623177e-03]
   [6.56469345e-01 7.94977069e-01 1.34521425e-02]
   [6.75795913e-01 2.74834007e-01 3.83842289e-02]
   [6.82180047e-01 1.01634908e+00 6.20636344e-03]
   [6.89419150e-01 5.16561568e-01 6.66916370e-04]]]]
[[[[5.1197338e-01 5.5373776e-01 7.1111941e-01]
   [4.3558729e-01 5.9714425e-01 5.7706481e-01]
   [4.4600210e-01 4.8162836e-01 6.7155808e-01]
   [4.

**How to get data**

In [13]:
right_eye = keypoints_with_scores[0][0][2]
np.array(right_eye[:-1]*[480,640]).astype(int)

array([209, 335])

In [15]:
shaped = np.squeeze(interpreter.get_tensor(output_details[0]['index'])*[480,640,1])
#print(shaped)
for kp in shaped:
    ky,kx,kp_conf = kp
    print(int(kx),int(ky),kp_conf)

379 236 0.7692314386367798
405 204 0.7363525629043579
335 209 0.47225403785705566
432 214 0.6654006242752075
289 228 0.7968537211418152
531 310 0.7863597869873047
215 331 0.8477886915206909
595 420 0.03343367576599121
190 422 0.03146308660507202
460 300 0.00731930136680603
292 383 0.0031796395778656006
259 208 0.009642869234085083
268 163 0.008072346448898315
618 315 0.01788806915283203
214 326 0.01835101842880249
651 245 0.009585678577423096
650 235 0.006071269512176514


## Draw Keypoints

In [16]:
def draw_keypoints(frame,keypoints,confidence):
    y,x,c = frame.shape
    shaped = np.squeeze(keypoints*[y,x,1])
    
    for kp in shaped:
        ky,kx,kp_conf = kp
        if kp_conf>confidence:
            cv2.circle(frame,(int(kx),int(ky)),4,(0,255,0),-1)
            

## Draw Edges

In [22]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}
#All connections

In [25]:
def draw_connections(frame,keypoints,edges,confidence):
    y,x,c = frame.shape
    shaped = np.squeeze(keypoints*[y,x,1]) 
    for edge,color in edges.items():
        p1,p2 = edge
        y1,x1,c1 = shaped[p1]
        y2,x2,c2 = shaped[p2]
        
        if(c1>confidence and c2>confidence):
            cv2.line(frame,(int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),2)